In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## Data Loading



In [ ]:
ls -lhtr /kaggle/input/tabular-playground-series-feb-2022

The dataset is about 1.2 GB and takes some time to load as csv, so I convert it to pickle file in order to save some loading time, everytime kernal restart. 
* the pickle size is 439 MB, which is way less than original and hence faster start. https://www.kaggle.com/khurana0112/tpsfeb2022train 
https://www.kaggle.com/khurana0112/uniquant-data-loading

* Another way to bypass this is to read the dataset using other tools, one such is dask. https://docs.dask.org/en/stable/dataframe.html but disadvantage there is to learn a new (similar) tool. 



In [ ]:
#%%time
#dna_train_tmp = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
#dna_test  = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")
#This cell takes 22.2 s


In [ ]:
%%time 
dna_train = pd.read_pickle("../input/d/khurana0112/tpsfeb2022train/train-2.pkl")


### * 22000 ms vs 716 ms, quite fast *

## EDA 

### Step 1
From following cells a quick inspenction of data shows following features: 
1. total 288 columns
2. except row_id and target all other are the features which can be used for inference. 
3. There are 10 unique target values, bacretial types. 
4. The frequency of the bacteria in the training dataset is very even, likely because its an scientific experiemnt or may be becuase they want to provide an even dataset, without any bias. 

In [ ]:
print (dna_train.shape, dna_train.columns)
dna_train.head(5)

In [ ]:
dna_train.target.unique() ## 10 unique target values, 

In [ ]:
dna_train.target.value_counts()

### Step 2 
- The genomic analysis technique has done some data compression and resulted in some data loss.
- In this technique, 10-mer snippets of DNA are sampled and analyzed to give the histogram of base count. 
- The bases are ATGC
- The distribution of the bases for 10-mers of sample (286 features) is from 0 to 10. Lets see the distribution of each of these for the full dataset. 
- ATGC distribution over the 286 features are exactly the same. There is no special feature/bias. 


In [ ]:
features = dna_train.columns[1:-1].tolist()
features[0:10]

In [ ]:
import re
for frs in features[0:10]:
    print (re.split('A|T|G|C',frs)[1:] )

In [ ]:
def getbases(feature):
    return re.split('A|T|G|C',frs)[1:]

In [ ]:
base_a=[]
base_t=[]
base_g=[]
base_c=[]
for frs in features:
    a_,t_,g_,c_ = (getbases(frs))
    base_a.append(int(a_)) ;    base_t.append(int(t_));   base_g.append(int(g_));   base_c.append(int(c_));
    


In [ ]:
df_bases = pd.DataFrame()
df_bases['base_a'] = pd.Series(base_a)
df_bases['base_t'] = pd.Series(base_t)
df_bases['base_g'] = pd.Series(base_g)
df_bases['base_c'] = pd.Series(base_c)



In [ ]:

df_bases.describe()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df_bases.base_a.value_counts().plot(title="base_a")
plt.figure()
df_bases.base_t.value_counts().plot(title="base_t")
plt.figure()
df_bases.base_g.value_counts().plot(title="base_g")
plt.figure()
df_bases.base_c.value_counts().plot(title="base_c")

In [ ]:
import seaborn as sns 
sns.pairplot(df_bases, vars=df_bases.columns[0:4]) # i can't plot the histplot and distplot for some reason, so easy way is to see the pair plot
